# Restricted Boltzmann Machine

## 0 - Import Dependencies

In [1]:
import numpy as np

## 1 - Helper Functions

In [2]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

## 2 - RBM Implementation

In [3]:
class RBM:
    
    def __init__(self, shape):
        
        self.vis_dim = shape[0] + 1
        self.hid_dim = shape[1] + 1
        
        self.weights = 0.2*(np.random.random(shape)-1)
        self.weights = np.insert(self.weights, 0, 0, axis=0)
        self.weights = np.insert(self.weights, 0, 0, axis=1)
        
    def forward_prop(self, data):
        
        hidden_output = np.dot(data, self.weights)
        hidden_probs = sigmoid(hidden_output)
        hidden_probs[:,0] = 1
        hidden_states = hidden_probs > np.random.rand(len(data), self.hid_dim)
        associations = np.dot(data.T, hidden_probs)
        return hidden_states, associations
    
    def back_prop(self, data):
        
        visible_output = np.dot(data, self.weights.T)
        visible_probs = sigmoid(visible_output)
        visible_probs[:,0] = 1
        visible_states = visible_probs > np.random.rand(len(data), self.vis_dim)
        associations = np.dot(data.T, visible_probs)
        return visible_states, associations
        
        
    def train(self, data, epochs=1000, lr=0.1):
        
        num_data = len(data)
        
        # Insert bias into first column
        data = np.insert(data, 0, 1, axis=1)
        
        # Forward propagation
        hidden_states, pos_assoc = self.forward_prop(data)
        
        # Backward propagation
        visible_states, neg_assoc = self.back_prop(hidden_states)
        
        # Weight update
        self.weights += lr * (((pos_assoc - neg_assoc.T)/num_data))

## 3 - Test Implementation

### 3.1 - Train RBM

In [4]:
# Initialise RBM
rbm = RBM((6,2))

# Initialise data
data1 = np.array([
    [1,0,1,0,1,0],
    [0,1,0,1,0,1],
    [1,0,0,0,1,0],
    [0,0,0,1,0,0],
    [0,0,1,0,0,0],
    [0,1,0,1,0,0]
])

data2 = np.array([
    [1,1,1,0,0,0],
    [0,0,0,1,1,1]
])

# Train RBM
for _ in range(1000):
    rbm.train(data2)

### 3.1 - Test RBM Forward Propagation

In [5]:
for pattern in ([1,1,1,0,0,0], [0,0,0,1,1,1]):
    
    left, right, neither, both = 0, 0, 0, 0

    for i in range(100000):
        x = np.array([pattern])
        x = np.insert(x, 0, 0, axis=1)
        hidden_states, _ = rbm.forward_prop(x)

        if hidden_states[0][1] == True and hidden_states[0][2] == False:
            left += 1

        elif hidden_states[0][1] == False and hidden_states[0][2] == True:
            right += 1

        elif hidden_states[0][1] == True and hidden_states[0][2] == True:
            both += 1

        else: neither += 1

    print({'Left':left, 'Right':right, 'Both':both, 'Neither':neither})

{'Both': 2, 'Neither': 0, 'Right': 99998, 'Left': 0}
{'Both': 0, 'Neither': 2, 'Right': 0, 'Left': 99998}


### 3.2 - Test RBM Backward Propagation

In [20]:
for _ in range(10):
    y = np.array([[True, False]])
    y = np.insert(y, 0, True, axis=1)
    visible_states, _ = rbm.back_prop(y)
    print([int(val) for val in visible_states[0][1:]])

[0, 0, 0, 1, 1, 1]
[0, 0, 0, 1, 1, 1]
[0, 0, 0, 1, 1, 1]
[0, 0, 0, 1, 1, 1]
[0, 0, 0, 1, 1, 1]
[1, 0, 0, 1, 1, 0]
[0, 0, 0, 1, 1, 1]
[0, 0, 0, 1, 1, 1]
[0, 0, 0, 1, 1, 1]
[0, 0, 0, 1, 1, 1]
